### 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助您：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(flyai_agent_in_action)`，即可执行下方代码

In [1]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate flyai_agent_in_action

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "flyai_agent_in_action" ]; then
    echo "✅ 当前单元格已成功激活到 flyai_agent_in_action 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的 Python 单元格将使用 Notebook 当前选择的 Jupyter 内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(flyai_agent_in_action)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(flyai_agent_in_action)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate flyai_agent_in_action"
fi

echo "=========================================" 

== Conda 环境检查报告 (仅针对当前 Bash 子进程) ==
✅ 当前单元格已成功激活到 flyai_agent_in_action 环境。
✅ 正在使用的环境路径: /workspace/envs/flyai_agent_in_action

💡 提示: 后续的 Python 单元格将使用 Notebook 当前选择的 Jupyter 内核。
   如果需要后续单元格也使用此环境，请执行以下操作:
   1. 检查 Notebook 右上角是否已选择 'python(flyai_agent_in_action)'。


In [2]:
# 2. 设置pip 为清华源
%pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list


Writing to /root/.config/pip/pip.conf
Note: you may need to restart the kernel to use updated packages.
global.index-url='https://pypi.tuna.tsinghua.edu.cn/simple'
:env:.target=''
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT

env: HF_ENDPOINT=https://hf-mirror.com
https://hf-mirror.com


In [4]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助您：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                  |
|:-------------|:----------------------------------------------------------------------|
| 操作系统     | Linux 5.15.0-126-generic                                              |
| CPU 信息     | Intel(R) Xeon(R) Platinum 8468 (48 physical cores, 192 logical cores) |
| 内存信息     | 2015.36 GB (Available: 1866.90 GB)                                    |
| GPU 信息     | No GPU found (checked nvidia-smi, lshw not found)                     |
| CUDA 信息    | 12.6                                                                  |
| Python 版本  | 3.12.11                                                               |
| Conda 版本   | conda 25.7.0                                                          |
| 物理磁盘空间 | Total: 2014.78 GB, Used: 651.70 GB, Free: 1260.66 GB                  |


# 动态断点 (Dynamic Breakpoints) - LangGraph 教程

## 什么是动态断点？

动态断点是 LangGraph 中一个强大的功能，允许在图的执行过程中**动态地**中断执行流程。与传统的静态断点不同，动态断点可以根据运行时的条件来决定是否中断，这为构建智能的人机协同系统提供了极大的灵活性。

# 动态断点 (Dynamic Breakpoints)

## 回顾：人机协同的动机

在前面的课程中，我们讨论了人机协同（Human-in-the-Loop）的三种主要动机：

### 1. 审批机制 (Approval)
- **功能**：中断智能体执行，向用户展示当前状态，允许用户决定是否接受某个操作
- **应用场景**：需要人工确认的重要决策，如金融交易、医疗诊断等

### 2. 调试功能 (Debugging)  
- **功能**：可以回退图的状态来重现或避免问题
- **应用场景**：开发调试、错误排查、性能优化

### 3. 状态编辑 (Editing)
- **功能**：允许修改图的状态
- **应用场景**：实时调整参数、修正错误、优化流程

## 传统断点 vs 动态断点

### 传统断点（静态断点）
- **设置时机**：在图编译时由开发者设置
- **特点**：固定的中断点，无法根据运行时条件改变
- **局限性**：缺乏灵活性，无法适应复杂的业务逻辑

### 动态断点（内部断点）
- **设置时机**：在节点内部根据条件动态触发
- **实现方式**：使用 `NodeInterrupt` 异常
- **优势**：
  1. **条件性中断**：基于开发者定义的逻辑条件
  2. **信息传递**：可以向用户传递中断原因
  3. **灵活控制**：根据运行时状态决定是否中断

## 本教程目标

我们将创建一个简单的图，演示如何根据输入长度动态触发中断。当输入超过5个字符时，图会抛出 `NodeInterrupt` 异常并暂停执行。

In [6]:
# 安装必要的依赖包
# 这个命令会安装 LangGraph 相关的核心包
# %%capture --no-stderr
# %pip install --quiet -U langgraph langchain_openai langgraph_sdk
%pip install --quiet langgraph==0.6.7 langchain_openai==0.3.32 langgraph_sdk==0.2.6

Note: you may need to restart the kernel to use updated packages.


In [8]:
# 导入必要的库
from IPython.display import Image, display
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.errors import NodeInterrupt
from langgraph.graph import START, END, StateGraph

# 定义图的状态结构
# TypedDict 用于定义状态的数据类型，确保类型安全
class State(TypedDict):
    input: str  # 输入字符串

# 第一步：简单的状态传递节点
def step_1(state: State) -> State:
    """
    第一步处理函数
    功能：接收状态并打印步骤信息，然后原样返回状态
    参数：state - 包含输入的状态字典
    返回：处理后的状态
    """
    print("---Step 1---")
    return state

# 第二步：包含动态断点的节点
def step_2(state: State) -> State:
    """
    第二步处理函数 - 演示动态断点
    功能：检查输入长度，如果超过5个字符则抛出 NodeInterrupt 异常
    参数：state - 包含输入的状态字典
    返回：处理后的状态（如果未中断）
    异常：NodeInterrupt - 当输入长度超过5个字符时抛出
    """
    # 动态断点逻辑：检查输入长度
    if len(state['input']) > 5:
        # 抛出 NodeInterrupt 异常，包含详细的错误信息
        raise NodeInterrupt(f"Received input that is longer than 5 characters: {state['input']}")

    print("---Step 2---")
    return state

# 第三步：最终处理节点
def step_3(state: State) -> State:
    """
    第三步处理函数
    功能：最终处理步骤，打印完成信息
    参数：state - 包含输入的状态字典
    返回：处理后的状态
    """
    print("---Step 3---")
    return state

# 构建图结构
print("正在构建 LangGraph...")
builder = StateGraph(State)

# 添加节点到图中
builder.add_node("step_1", step_1)  # 添加第一步节点
builder.add_node("step_2", step_2)  # 添加第二步节点（包含动态断点）
builder.add_node("step_3", step_3)  # 添加第三步节点

# 定义节点之间的连接关系
builder.add_edge(START, "step_1")      # 从开始到第一步
builder.add_edge("step_1", "step_2")   # 从第一步到第二步
builder.add_edge("step_2", "step_3")   # 从第二步到第三步
builder.add_edge("step_3", END)        # 从第三步到结束

# 设置内存检查点
# MemorySaver 用于保存图的状态，支持中断和恢复
memory = MemorySaver()

# 编译图并添加内存检查点
graph = builder.compile(checkpointer=memory)

print("图构建完成！")
print("图结构：START -> step_1 -> step_2 -> step_3 -> END")
print("注意：step_2 包含动态断点逻辑")

# 显示图的可视化结构
print("\n正在生成图的可视化...")
# 展示图结构
# 图可视化
print("图可视化：")
# 方案1：尝试使用 Pyppeteer 本地渲染（推荐）
try:
    # 可视化：通过 Mermaid 渲染图结构
    display(Image(graph.get_graph().draw_mermaid_png()))
    print("✅ 图渲染成功！")
except Exception as e:
    print(f"❌ Pyppeteer 渲染失败: {e}")
    
    # 方案2：显示 Mermaid 文本格式
    print("\n📝 图结构（Mermaid 文本格式）：")
    print("=" * 50)
    mermaid_text = graph.get_graph().draw_mermaid()
    print(mermaid_text)
    print("=" * 50)
    
    # 方案3：显示图的节点和边信息
    print("\n🔗 图结构信息：")
    print("节点:", list(graph.get_graph().nodes.keys()))
    print("边:", list(graph.get_graph().edges))
    
    # 方案4：提供手动渲染说明
    print("\n💡 手动渲染说明：")
    print("1. 复制上面的 Mermaid 文本")
    print("2. 访问 https://mermaid.live/")
    print("3. 粘贴文本到编辑器中查看图形")
    print("4. 或者使用支持 Mermaid 的 Markdown 编辑器")

正在构建 LangGraph...
图构建完成！
图结构：START -> step_1 -> step_2 -> step_3 -> END
注意：step_2 包含动态断点逻辑

正在生成图的可视化...
图可视化：
❌ Pyppeteer 渲染失败: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 503.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

📝 图结构（Mermaid 文本格式）：
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	step_1(step_1)
	step_2(step_2)
	step_3(step_3)
	__end__([<p>__end__</p>]):::last
	__start__ --> step_1;
	step_1 --> step_2;
	step_2 --> step_3;
	step_3 --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc


🔗 图结构信息：
节点: ['__start__', 'step_1', 'step_2', 'step_3', '__end__']


## 测试动态断点功能

现在让我们用一个超过5个字符的输入来测试图，看看动态断点是如何工作的。

In [9]:
# 准备测试数据
# 使用一个超过5个字符的输入来触发动态断点
initial_input = {"input": "hello world"}  # 11个字符，会触发断点
thread_config = {"configurable": {"thread_id": "1"}}  # 线程配置，用于状态管理

print("开始执行图...")
print(f"输入内容: '{initial_input['input']}' (长度: {len(initial_input['input'])} 字符)")
print("预期结果: 在 step_2 处触发动态断点")
print("-" * 50)

# 运行图直到第一次中断
# stream_mode="values" 表示只返回状态值，不返回其他元数据
for event in graph.stream(initial_input, thread_config, stream_mode="values"):
    print(f"状态更新: {event}")

print("-" * 50)
print("图执行被中断！")

开始执行图...
输入内容: 'hello world' (长度: 11 字符)
预期结果: 在 step_2 处触发动态断点
--------------------------------------------------
状态更新: {'input': 'hello world'}
---Step 1---
状态更新: {'input': 'hello world'}
--------------------------------------------------
图执行被中断！


/workspace/tmp/ipykernel_8778/2122628644.py:36: LangGraphDeprecatedSinceV10: NodeInterrupt is deprecated. Please use `langgraph.types.interrupt` instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  raise NodeInterrupt(f"Received input that is longer than 5 characters: {state['input']}")


## 检查中断后的图状态

当图被中断后，我们可以检查当前的状态，看看下一个要执行的节点是什么。


In [10]:
# 获取当前图的状态
state = graph.get_state(thread_config)

print("当前图状态信息:")
print(f"下一个要执行的节点: {state.next}")
print(f"当前状态值: {state.values}")
print(f"是否有中断: {len(state.tasks) > 0 and any(task.interrupts for task in state.tasks)}")

# 检查中断详情
if state.tasks:
    for task in state.tasks:
        if task.interrupts:
            print(f"中断信息: {task.interrupts}")

当前图状态信息:
下一个要执行的节点: ('step_2',)
当前状态值: {'input': 'hello world'}
是否有中断: True
中断信息: (Interrupt(value='Received input that is longer than 5 characters: hello world', id='placeholder-id'),)


## 查看中断详情

我们可以看到中断信息被记录在状态中，这包含了中断的原因和上下文信息。

In [11]:
# 查看任务详情，包括中断信息
print("任务详情:")
for i, task in enumerate(state.tasks):
    print(f"任务 {i+1}:")
    print(f"  - 任务ID: {task.id}")
    print(f"  - 节点名称: {task.name}")
    print(f"  - 错误信息: {task.error}")
    print(f"  - 中断信息: {task.interrupts}")
    print(f"  - 状态: {task.state}")
    print()

任务详情:
任务 1:
  - 任务ID: 7241a725-8b43-b412-21a5-09f8ddc6fce6
  - 节点名称: step_2
  - 错误信息: None
  - 中断信息: (Interrupt(value='Received input that is longer than 5 characters: hello world', id='placeholder-id'),)
  - 状态: None



## 尝试恢复图执行

我们可以尝试从断点处恢复图的执行。但是，如果状态没有改变，图会重新执行同一个节点，导致无限循环！

In [12]:
# 尝试恢复图执行（不改变状态）
print("尝试恢复图执行（不改变状态）...")
print("注意：这会导致重新执行 step_2，再次触发中断！")
print("-" * 50)

for event in graph.stream(None, thread_config, stream_mode="values"):
    print(f"状态更新: {event}")

print("-" * 50)
print("图再次被中断！")

尝试恢复图执行（不改变状态）...
注意：这会导致重新执行 step_2，再次触发中断！
--------------------------------------------------
状态更新: {'input': 'hello world'}
--------------------------------------------------
图再次被中断！


/workspace/tmp/ipykernel_8778/2122628644.py:36: LangGraphDeprecatedSinceV10: NodeInterrupt is deprecated. Please use `langgraph.types.interrupt` instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  raise NodeInterrupt(f"Received input that is longer than 5 characters: {state['input']}")


In [13]:
# 再次检查状态，确认仍在 step_2
state = graph.get_state(thread_config)
print(f"当前状态: 仍在 {state.next} 节点")
print("这证明了如果不改变状态，图会一直重复执行同一个节点！")

当前状态: 仍在 ('step_2',) 节点
这证明了如果不改变状态，图会一直重复执行同一个节点！


## 修改状态以解决中断

现在我们需要修改状态来解决中断问题。我们可以将输入改为5个字符或更少，这样就不会触发动态断点了。

In [14]:
# 更新状态：将输入改为短字符串，避免触发断点
print("正在更新状态...")
print("原输入: 'hello world' (11字符)")
print("新输入: 'hi' (2字符)")

# 使用 update_state 方法修改图的状态
result = graph.update_state(
    thread_config,
    {"input": "hi"},  # 新的输入，只有2个字符
)

print(f"状态更新结果: {result}")
print("现在可以继续执行图了！")

正在更新状态...
原输入: 'hello world' (11字符)
新输入: 'hi' (2字符)
状态更新结果: {'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09dafc-6e20-62ef-8002-60b2703d7bef'}}
现在可以继续执行图了！


In [15]:
# 继续执行图，现在应该能正常完成
print("继续执行图...")
print("预期结果: 所有步骤都能正常执行，不会触发中断")
print("-" * 50)

for event in graph.stream(None, thread_config, stream_mode="values"):
    print(f"状态更新: {event}")

print("-" * 50)
print("图执行完成！")

继续执行图...
预期结果: 所有步骤都能正常执行，不会触发中断
--------------------------------------------------
状态更新: {'input': 'hi'}
---Step 2---
状态更新: {'input': 'hi'}
---Step 3---
状态更新: {'input': 'hi'}
--------------------------------------------------
图执行完成！


## 总结

### 动态断点的核心概念

1. **条件性中断**：根据运行时条件决定是否中断
2. **信息传递**：通过 `NodeInterrupt` 传递中断原因
3. **状态管理**：需要修改状态才能解决中断问题

### 关键学习点

1. **NodeInterrupt 异常**：用于在节点内部触发动态中断
2. **状态更新**：使用 `update_state` 方法修改图状态
3. **中断恢复**：修改状态后可以继续执行图
4. **Studio 集成**：通过 SDK 在 Studio 环境中使用动态断点

### 实际应用场景

- **内容审核**：当检测到敏感内容时中断，等待人工审核
- **质量控制**：当输出质量不达标时中断，要求重新生成
- **用户确认**：当需要用户确认重要操作时中断
- **错误处理**：当检测到异常情况时中断，等待处理

### 最佳实践

1. **明确中断条件**：确保中断条件清晰且必要
2. **提供有用信息**：在 `NodeInterrupt` 中包含足够的上下文信息
3. **状态设计**：确保状态可以被合理修改以解决中断
4. **用户体验**：考虑用户如何理解和处理中断情况